In [17]:
from utils import *
import keras
import h5py

In [2]:
Xorg, Y = read_input('inputs.txt')


# inputs.txt contains one dataset
# X is training data; Y is validation??

read file ./TTbar_set1.h5
finish reading files


In [4]:
Y = Y / -1

Xi, Xc1, Xc2 = preProcessing(Xorg, 1)
Xc = [Xc1, Xc2]

emb_input_dim = {i:int(np.max(Xc[i][0:1000])) + 1 for i in range(2) }
print(emb_input_dim)

# Prepare training/val data
Yr = Y
Xr = [Xi] + Xc

# This is copied from train.py
# Question: do we know the dimensoins genercially (before reading the file)

{0: 13, 1: 3}


In [5]:
with open('inputs.txt') as f:
    sets = f.readlines()

In [6]:
sets[0]

'./TTbar_set1.h5'

In [8]:
partition= {}
partition['train'] = [f'{sets[0]}']
partition['validation'] = [f'{sets[0]}']
i = 0
for set in sets:
    if i != 0:
        partition['train'] += [f'{set}']
        partition['validation'] += [f'id-{set}']
    i+=1
    


In [9]:
labels = {}
labels[f'{sets[0]}'] = [0]
i = 0
for set in sets:
    if i != 0:
        labels[f'set'] = i

# sets up partitions and labels per the example
# Question: is the goal of this data generator to give the model one file at a time or a subset (batch size) of a file at one time 

In [26]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=1024, dim=(1,100,8), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def read_file(self, fname):
        #print("read file", fname)
        h5f = h5py.File( fname, 'r')
        X = h5f['X'][:]
        Y = h5f['Y'][:]
        #print("finish reading files")
        A = X[:, :, :]
        B = Y[:, :]
        return A, B

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = self.read_file(ID)

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)
    
# copied from the example; added a method to read any given h5 file
# I think something is wrong with dimenions ive set, but i'm lacking an intuision (see error in line 29)

In [27]:
params = {'dim': (1,100,8),
          'batch_size': 1024,
          'n_classes': 6,
          'n_channels': 1,
          'shuffle': True}

In [28]:
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

In [29]:
training_generator.__getitem__(0)

read file ./TTbar_set1.h5
finish reading files


ValueError: could not broadcast input array from shape (161837,100,8) into shape (161837)